# Imports

In [4]:
import re
import random

from midiutil import MIDIFile

import pygame
import pygame.mixer
from time import sleep

pygame 2.1.3.dev8 (SDL 2.0.22, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Utils

In [5]:
def mergeDict(dict1, dict2):
    dict3 = {**dict1, **dict2}
    for key in dict3:
        if key in dict1 and key in dict2:
            dict3[key] = [dict1[key] , dict2[key]]
    return dict3

In [6]:
def unfoldDict(dict1):
    dict2 = {}
    for k,v in dict1.items():
        if type(v)==list:
            for vi in v:
                dict2[vi]=k
        else:
            dict2[v]==k
    return dict2

# Vars

### Alturas
C0 = 0, C#0 = 1, ..., C1 = 12, ...

### Duraciones (% del beat)
En 4/4: blanca = 2, negra = 1, corchea = 0.5,...


# Processes

## Pitch conversor

In [7]:
# INITIAL VARS

# Usual names with sharps
pitches1 = ["c","c+","d","d+","e","f","f+","g","g+","a","a+","b"]

# Additional names
pitches2 = ['b+','d-','d','e-','f-','e+','g-','g','a-','a','b-','c-']

# Complete dicts
pitch_number_2_name = mergeDict(dict(enumerate(pitches1)),dict(enumerate(pitches2)))

pitch_name_2_number = unfoldDict(pitch_number_2_name)

# FUNCTIONS

def to_name(pitch_number):
    if type(pitch_number)!=str:
        return [v+str(pitch_number//12) for v in pitch_number_2_name[pitch_number%12]]
    else:
        return pitch_number

def to_number(pitch_name):
    if type(pitch_name)==str:
        pitch_number = 12*int(re.findall(r"\d+",pitch_name)[0])+pitch_name_2_number[re.findall(r"\D+",pitch_name)[0]]
        return pitch_number
    else:
        return pitch_name
    
# EXAMPLES
    
pitch_ex = 61
pitch_ex_n1 = "c+4"
pitch_ex_n2 = "d-4"

print("The {}th pitch is {}".format(pitch_ex,to_name(pitch_ex)))
print("The {} is the {}th pitch".format(pitch_ex_n1,to_number(pitch_ex_n1)))
print("The {} is the {}th pitch".format(pitch_ex_n2,to_number(pitch_ex_n2)))

pitch_ex_n1 = "c4"
pitch_ex_n2 = "c-4"

print("The {} is the {}th pitch".format(pitch_ex_n1,to_number(pitch_ex_n1)))
print("The {} is the {}th pitch".format(pitch_ex_n2,to_number(pitch_ex_n2)))

The 61th pitch is ['c+5', 'd-5']
The c+4 is the 49th pitch
The d-4 is the 49th pitch
The c4 is the 48th pitch
The c-4 is the 59th pitch


## INTERVALS

In [9]:
def pitch_after_interval(note, interval, int_type = 1):
    '''
    note = str as in pitch_name_2_number dict variable keys
    interval = str as in intervals dict variable
    int_type = int +1/-1 for ascendent/descendent interval type
    '''  
      
    # Usual names and semitones for intervals
    intervals = {k:i for i,k in enumerate(['1','b2','2','b3','3','4','#4','5','b6','6','b7','7'])}
    # Extended list
    intervals = {**intervals,**{'b1':-1,'#1':1,'#2':3,'#3':5,'b4':4,'b5':6,'#5':8,'#6':10,'#7':12}}
    
    # Notes
    notes = 'abcdefg'
    
    # Possible final notes
    interval_num = int(re.findall(r'\d+',interval)[0])
    interval_num_red = interval_num%7+7*(interval_num%7==0)
    if interval[0] in ['b','#']:
        interval_red = interval[0]+str(interval_num_red)
    else:
        interval_red = str(interval_num_red)
    
    final_note_list = to_name(to_number(note)+int_type*(intervals[interval_red]+12*(interval_num//8)))
    final_note_name = notes[(notes.index(note[0])+int_type*(interval_num%7-1))%7]
    
    # Final note
    for final_note in final_note_list:
        if final_note_name in final_note:
            return final_note
    
    if final_note_list[0]==final_note_list[1]:
        return final_note_list[0]

# EXAMPLES

print(pitch_after_interval('c4','b2',1))
print(pitch_after_interval('c4','b2',-1))

d-4
b3


### Funciona bien con ascendentes

In [10]:
def interval_between_pitches(note_1, note_2):
        
    # Initial computations
    note_1_num, note_2_num = to_number(note_1), to_number(note_2)
    int_num = note_2_num - note_1_num
    int_num_oct = int_num//12
    int_num_red = int_num%12
    
    # Interval type
    int_type = 1-2*(int_num < 0)
    
    # Usual names and semitones for intervals
    intervals = {k:i for i,k in enumerate(['1','b2','2','b3','3','4','#4','5','b6','6','b7','7'])}
    # Extended list
    intervals = {**intervals,**{'b1':-1,'#1':1,'#2':3,'#3':5,'b4':4,'b5':6,'#5':8,'#6':10,'#7':12}}
    
    # Possible intervals
    possible_intervals = []
    for i,st in intervals.items():
        if st == int_num_red:
            poss_int_num_red = re.findall(r'\D+',i)
            poss_int_num = str(int(i[len(poss_int_num_red):])+int_num_oct*7)
            if len(poss_int_num_red)!=0:
                possible_intervals.append(i[0]+poss_int_num)
            else:
                possible_intervals.append(poss_int_num)
                
    # Loop
    for poss_int in possible_intervals:
        final_note = pitch_after_interval(note_1,poss_int,int_type)       
        if note_2 == final_note:
            return poss_int
    return possible_intervals

# EXAMPLES

print(interval_between_pitches('c4','c5'))
print(interval_between_pitches('c4','d-5'))
print(interval_between_pitches('c4','f+5'))

8
b9
#11


## CHORDS

In [11]:
chords = {'dim':['b3','b5'],'m':['b3','5'],'M':['3','5'],'aum':['3','#5'],
          'dim7':['b3','b5','6'],'m7b5':['b3','b5','b7'],'m7':['b3','5','b7'],
          'mM7':['b3','5','7'],'7':['3','5','b7'],'7sus4':['4','5','b7'],
          'M7':['3','5','7'],'aum7':['3','#5','b7'],'M7#5':['3','#5','7']}

def chord(root='c4', chord_type='M'):
            
    note_chords = [root]
    
    for interval in chords[chord_type]:
        note_chords.append(pitch_after_interval(root, interval))
    
    return note_chords

# EXAMPLE

print(chord('c+4','aum'))
print(chord('d5','M'))
print(chord('e3','M7'))

['c+4', 'e+4', 'a4']
['d5', 'f+5', 'a5']
['e3', 'g+3', 'b3', 'd+4']


## SCALES

In [12]:
# INITIAL VARS

pentatonic = {"major": ['2', '3', '5', '6'],
              "minor": ['b3', '4', '5', 'b7']}
hexatonic = {"whole_tone": ['2', '3', '#4', '#5', '#6']}
lydian = {"lydian": ['2', '3', '#4', '5', '6', '7'],
          "lydian_dom": ['2', '3', '#4', '5', '6', 'b7'],
          "lydian_augm": ['2', '3', '#4', '#5', '6', '7']}
major = {"major": ['2', '3', '4', '5', '6', '7'],
         "major_harm": ['2', '3', '4', '5', 'b6', '7']}
mixolydian = {"mixolydian": ['2', '3', '4', '5', '6', 'b7'],
              "mixolydian_b9": ['b2', '3', '4', '5', '6', 'b7'],
              "mixolydian_b9b13": ['b2', '3', '4', '5', 'b6', 'b7']}
dorian = {"dorian": ['2', 'b3', '4', '5', '6', 'b7'],
          "dorian_b9": ['b2', 'b3', '4', '5', '6', 'b7']}
minor = {"aeolian": ['2', 'b3', '4', '5', 'b6', 'b7'],
         "minor_harm": ['2', 'b3', '4', '5', 'b6', '7'],
         "minor_mel": ['2', 'b3', '4', '5', '6', '7']}
phrygian = {"phrygian": ['b2', 'b3', '4', '5', 'b6', 'b7']}
locrian = {"locrian": ['b2', 'b3', '4', 'b5', 'b6', 'b7'],
           "locrian_n9": ['2', 'b3', '4', 'b5', 'b6', 'b7'],
           "altered": ['b2', 'b3', 'b4', 'b5', 'b6', 'b7']}
octatonic = {"hs_ws": ['b2', 'b3', '3', '#4', '5', '6', 'b7'],
             "ws_hs": ['2', 'b3', '4', 'b5', 'b6', '6', '7']}
chromatic = {"chromatic": ['b2', '2', 'b3',
                           '3', '4', '#4', '5', 'b6', '6', 'b7', '7']}

scales_dict = {**pentatonic, **hexatonic, **lydian, **major, **mixolydian,
               **dorian, **minor, **phrygian, **locrian, **octatonic, **chromatic}

# FUNCTION


def get_scale(root='c4', scale='major', scale_type=1):
    '''
    root = str for the root of the scale as in pitch_name_2_number dict variable keys
    scale = str for the name of the scale as in scales_dict variable
    scale_type = +1/-1 for ascendente/descendent
    '''
    scale_notes = [root]
    if scale_type == 1:
        for degree in scales_dict[scale]:
            scale_notes.append(pitch_after_interval(root, degree))
    else:
        for degree in scales_dict[scale][::-1]:
            scale_notes.append(pitch_after_interval(
                pitch_after_interval(root, degree), '8', -1))
    return scale_notes

# EXAMPLES


print(get_scale('c5', 'whole_tone'))
print(get_scale('d4', 'major', -1))
print(get_scale('e3', 'lydian_dom'))


['c5', 'd5', 'e5', 'f+5', 'g+5', 'a+5']
['d4', 'c+4', 'b3', 'a3', 'g3', 'f+3', 'e3']
['e3', 'f+3', 'g+3', 'a+3', 'b3', 'c+4', 'd4']


## DICTATES

In [13]:
def dictate(root='c4',scale='chromatic',length=16,
            durations=[0.5,1,1.5,2,3,4],tempo=75,
            midi_fn='dictado.mid'):
    
    track    = 0
    channel  = 0
    time     = 0    
    volume   = 127
    
    available_pitches = get_scale(root,scale)
    pitches = [root]+random.choices(available_pitches,k=length)
    durations = [4]+random.choices(durations,k=length)
    
    MyMIDI = MIDIFile(1)
    MyMIDI.addTempo(track, time, tempo)
    
    for i, pitch in enumerate(pitches):
        MyMIDI.addNote(track, channel, to_number(pitch), time, durations[i], volume)
        time += durations[i]
    
    file_name = "prueba_midi_dictado"

    with open(file_name, "wb") as output_file:
        MyMIDI.writeFile(output_file)

    pygame.init()
    pygame.mixer.init()
    pygame.mixer.music.load(file_name)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        sleep(1)
    print("Done!")

# EXAMPLE    

dictate()

Done!


In [14]:
def play_melody(pitches, duration=1,tempo=75, midi_fn='play_melody.mid'):
    
    track    = 0
    channel  = 0
    time     = 0    
    volume   = 127
    
    MyMIDI = MIDIFile(1)
    MyMIDI.addTempo(track, time, tempo)
    
    for i, pitch in enumerate(pitches):
        MyMIDI.addNote(track, channel, to_number(pitch), time, duration, volume)
        time += duration
    
    file_name = "prueba_midi_melodía"   

    with open(file_name, "wb") as output_file:
        MyMIDI.writeFile(output_file)

    pygame.init()
    pygame.mixer.init()
    pygame.mixer.music.load(file_name)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        sleep(1)
    print("Done!")

# EXAMPLE    

play_melody([*chord('c4','m7b5'),*get_scale('d6','major',-1)])


Done!


In [15]:
def play_chords(chords_list, duration=4,tempo=75, midi_fn='play_chords.mid'):
    
    track    = 0
    channel  = 0
    time     = 0    
    volume   = 127
    
    MyMIDI = MIDIFile(1)
    MyMIDI.addTempo(track, time, tempo)
    
    for chord in chords_list:
        for i, pitch in enumerate(chord):
            MyMIDI.addNote(track, channel, to_number(pitch), time, duration, volume)
        time += duration
    
    file_name = "prueba_midi_acordes"

    with open(file_name, "wb") as output_file:
        MyMIDI.writeFile(output_file)

    pygame.init()
    pygame.mixer.init()
    pygame.mixer.music.load(file_name)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        sleep(1)
    print("Done!")

# EXAMPLE 
    
play_chords([chord('c4','M7'),chord('e4','7'),chord('a4','m7'),
            chord('d4','m7'),chord('g4','7sus4'),chord('g4','7'),
            chord('c4','M7')])

Done!
